In [1]:
import requests
from bs4 import BeautifulSoup,SoupStrainer
import pandas as pd
from IPython.display import Markdown, display,HTML,Image
from selenium import webdriver
import time

In [2]:
# get the solution links
pages = ['https://www.janestreet.com/puzzles/archive/index.html',
         'https://www.janestreet.com/puzzles/archive/page2/index.html',
         'https://www.janestreet.com/puzzles/archive/page3/index.html',
         'https://www.janestreet.com/puzzles/archive/page4/index.html',
         'https://www.janestreet.com/puzzles/archive/page5/index.html',
         'https://www.janestreet.com/puzzles/archive/page6/index.html',
         'https://www.janestreet.com/puzzles/archive/page7/index.html',
         'https://www.janestreet.com/puzzles/archive/page8/index.html',
         'https://www.janestreet.com/puzzles/archive/page9/index.html',
         'https://www.janestreet.com/puzzles/archive/page10/index.html',
         'https://www.janestreet.com/puzzles/archive/page11/index.html',
         'https://www.janestreet.com/puzzles/archive/page12/index.html']  
links = ['https://www.janestreet.com/puzzles/current-puzzle/']
for url in pages:
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    links += ['https://www.janestreet.com/'+i['href'] for i in soup.find_all('a', {'class' :'solution-link'})]   

In [3]:
# scrape the names of solvers
participants = []
order = 1
driver = webdriver.Firefox()
for url in links:
    driver.get(url)
    time.sleep(5)
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource, 'html.parser')
    participants += [[str(url).split("/")[-1],i.split(" (")[0],order] for i in soup.find_all('p', {'class' :'correct-submissions'})[0].stripped_strings]
    order +=1

In [10]:
# Work out the scores with an offset. Count as attempted all those since the name first appeared on a leaderboard.
# No adjustments for similar names etc.
# Borrowed the offset idea from this (much nicer) tableau league table 
# https://public.tableau.com/app/profile/heidi.stockton/viz/PuzzlesofJaneStreet/JaneStreet

newcomer_offset = 3

df = pd.DataFrame(participants,columns = ['puzzle','name','order'])
df2 = df[['name','order']].groupby('name').agg({'name':['count'],'order': ['max']})
df2.columns = [' '.join(col).strip() for col in df2.columns.values]
df2.columns = ['Solved','Attempted']
df2['score']=df2['Solved']/(df2['Attempted']+ newcomer_offset)
df2.sort_values('score',ascending=False)[:20]

,Solved,Attempted,score
name,,,
Evan Semet,26,26,0.896552
Karl Mahlburg,44,50,0.830189
Senthil Rajasekaran,49,61,0.765625
Gareth Owen,32,41,0.727273
Lazar Ilic,29,38,0.707317
Aaditya Raghavan,33,44,0.702128
Calvin Pozderac,52,74,0.675325
Arthur Bright,12,15,0.666667
Blaine Hill,6,6,0.666667


In [5]:
df2.sort_values('Solved',ascending=False)[:20]

,Solved,Attempted,score
name,,,
Calvin Pozderac,52,74,0.666667
Senthil Rajasekaran,49,61,0.753846
Sean Egan,47,68,0.652778
Karl Mahlburg,44,50,0.814815
Keith Schneider,41,65,0.594203
Heidi Stockton,39,85,0.438202
Aaditya Raghavan,33,44,0.687500
Sébastien Geeraert,32,59,0.507937
Gareth Owen,32,41,0.711111
